In [1]:
import numpy as np
T = 50

We shall generate the STATE-SPACE variables: $N_t, L_t, Y_t$.

We shall generate the ACTION variables: $A_t$.

We shall also generate nuisance variables $B_t, X_t$.

Intermediate variables are $Z_t, \; \mu_t, \; K_t$.

Length of the trajectory is determined by $L_t$ = discontinuation indicator 
and $D_t$ = death indicator.

t = 0, ... T - 1

Generation of $B_t, X_t, \; t = 0, \dots T - 1$

In [2]:
sig_epsilon = 0.5
B_t = np.zeros(T)
X_t = np.zeros(T)
for i in range(1, T):
  B_t[i] = np.random.normal(loc = B_t[i - 1], scale = sig_epsilon)/np.sqrt(1 + sig_epsilon**2)
for i in range(1, T):
  X_t[i] = np.random.normal(loc = X_t[i - 1], scale = sig_epsilon)/np.sqrt(1 + sig_epsilon**2)

Assume $D_0 = 0, \; Y_0 \sim N(7.7, 1), \; L_0 = 0, \; A_0 = 0, \; K_0 = 0, \; \mu_0 = 1, \; R_0 = 0.5$.

We initialize the arrays for $D_t, N_t, L_t, \; Y_t, \; A_t, \; \mu_t \; K_t, \; R_t$.



Reward $R_t \equiv R_t(Y_{t - 1}, Y_t, D_t) = \exp \left( - \frac{Y_t + Y_{t - 1}}{15} \right) \boldsymbol{1}_{D_t = 0}$


MEDICINE ENCODING:

No change = 0

Metformin = 1

Sulfonylurea = 2

Glitazone = 3

Insulin = 4

Define the logistic function:

In [3]:
def expit(x):
  return 1 - (1/(1 + np.exp(x)))

def expit_lam(x, lam = 1):
  return 1 - (1/(1 + np.exp(lam*x)))

In [4]:
def gen_tracks(track_length, track_count):
  out = [None]*track_count
  for i in range(track_count):

    # monitoring length of tracks  
    counter = 0

    # initializing arrays
    d_array = np.array([0])
    n_array = np.array([0])
    l_array = np.array([0])
    y_array = np.array([np.random.normal(loc = 7.7, scale = 1)])
    a_array = np.array([0])
    mu_array = np.array([7.7])
    k_array = np.array([0])
    r_array = np.array([0.5])

    while counter < track_length and d_array[-1] == 0:
    # updating N_t, A_t
      n_old = n_array[-1]
      l_old = l_array[-1]
      y_old = y_array[-1]
      if n_old == 4:
        n_new = n_old
        a_new = n_new
      else:
        if y_old < 7:
          n_new = n_old
          a_new = 0
        elif y_old >= 8:
          n_new = n_old + 1
          a_new = n_new
        else:
          z = np.random.binomial(n = 1, p = expit(-0.2*y_old + 0.2*n_old + 0.5*l_old)) # this is the Z_t variable when A1c level is between 7 and 8
          if z == 1:
            n_new = n_old
            a_new = 0
          else:
            n_new = n_old + 1
            a_new = n_new
      n_array = np.append(n_array, n_new)
      a_array = np.append(a_array, a_new)

      # updating L_t
      a_old = a_array[-2]
      if a_old == 4:
        l_new = np.random.binomial(n = 1, p = 0.35)
      elif a_old == 1 or a_old == 2 or a_old == 3:
        l_new = np.random.binomial(n = 1, p = 0.2)
      else:
        l_new = 0
      l_array = np.append(l_array, l_new)

      # updating K_t
      if y_array[-1] >= 7 and n_array[-2] < 4 and a_array[-2] !=0 and l_array[-2] == 0:
        k_new = 1
      else:
        k_new = 0
      k_array = np.append(k_array, k_new)

      # updating mu_t
      tau = np.array([.14, 0.2, 0.02]) # percentage reduction of A1c
      a_new = a_array[-1]
      mu_old = mu_array[-1]
      k_new = k_array[-1]
      if a_new == 0:
        mu_new = mu_old
      elif a_new == 1 or a_new == 4:
        mu_new = mu_old*(1 - k_new*tau[0])
      elif a_new == 2:
        mu_new = mu_old*(1 - k_new*tau[1])
      elif a_new == 3:
        mu_new = mu_old*(1 - k_new*tau[2])
      mu_array = np.append(mu_array, mu_new)

      # updating Y_t
      y_old = y_array[-1]
      mu_new = mu_array[-1]
      mu_old = mu_array[-2]
      y_new = (y_old - mu_old + np.random.normal(loc = 0, scale = sig_epsilon))/np.sqrt(1 + sig_epsilon**2) + mu_new
      y_array = np.append(y_array, y_new)

      # updating D_t
      if y_array[-1] > 7:
        d_new = np.random.binomial(n = 1, p = expit(-7.5 + 0.08*y_array[-1]**2 + 0.5*n_array[-1]))
      else:
        d_new = np.random.binomial(n = 1, p = expit(-7.5 + 0.5*n_array[-1]))
      d_array = np.append(d_array, d_new)

      # updating R_t
      if d_array[-1] == 0:
        r_new = np.exp(-(y_array[-1] + y_array[-2])/15)
      else:
        r_new = 0
      r_array = np.append(r_array, r_new)

      counter = counter + 1

    out[i] = [r_array, y_array, d_array]
  return out

data = list of tracks of length atmost $51$

In [5]:
data = gen_tracks(track_length = 30, track_count = 100)

Finding out how many tracks have deaths, given data:

In [6]:
death_ratio = sum([len(a[0]) < 31 for a in data])/len(data)
death_ratio

0.56

In [7]:
data[0]

[array([0.5       , 0.39353731, 0.39180165, 0.39026672, 0.3926463 ,
        0.37860768, 0.        ]),
 array([7.12256836, 6.86612275, 7.18887062, 6.92500236, 7.09768863,
        7.4711325 , 8.13650042]),
 array([0, 0, 0, 0, 0, 0, 1])]

We assume states are $(Y_t, D_t)$ and actions are $A_t := \boldsymbol{1}_{N_t \neq N_{t - 1}}$.

We need to estimate $\overline{Q} \left( Y_{t - 1}, D_{t - 1} = 0, A_t, Y_t, D_t = 1 \right)$ properly for my method to work

Let us look at the kernel function in our setup. Since it describes the covariance between two features, we first figure out the exact features we want to use. Let $x_1 = \left( y_r, y_{r - 1}, d_r, d_{r - 1} \right), x_2 = \left( y_s, y_{s - 1}, d_s, d_{s - 1} \right)$ be two features. Then we can model $K\left(x_1, x_2 \right) = \cos \left( \theta_2 \right). RBF(y_1 - y_2; \theta_2)$ when $\left( d_r, d_{r - 1} \right) = (0, 0), \; \left( d_s, d_{s - 1} \right) = (0, 1)$ or vice versa, and as $K\left(x_1, x_2 \right) = RBF(x_1 - x_2; \theta_2)$ when $\left( d_r, d_{r - 1} \right) = \left( d_s, d_{s - 1} \right) = (0, 0), (0, 1)$. Observe that other death value pairs are not observable in the data, which allows us to use just one extra hyper-parameter $\theta_2$ other than the RBF hyper-parameter $\theta_1$.

In [ ]:
cc = [list(a) for a in zip(data[0][1][:-1], data[0][1][1:])]

In [ ]:
dd = [list(a) for a in zip(data[0][2][:-1], data[0][2][1:])]

In [ ]:
np.concatenate((cc, dd), axis = 1)

array([[6.13743427, 6.73536367, 0.        , 0.        ],
       [6.73536367, 7.70252165, 0.        , 0.        ],
       [7.70252165, 8.3341674 , 0.        , 0.        ],
       [8.3341674 , 6.47477442, 0.        , 0.        ],
       [6.47477442, 6.77222273, 0.        , 0.        ],
       [6.77222273, 6.1490415 , 0.        , 0.        ],
       [6.1490415 , 6.59553235, 0.        , 0.        ],
       [6.59553235, 6.20965588, 0.        , 0.        ],
       [6.20965588, 5.96347739, 0.        , 0.        ],
       [5.96347739, 6.20406605, 0.        , 0.        ],
       [6.20406605, 6.38957805, 0.        , 0.        ],
       [6.38957805, 6.78035941, 0.        , 0.        ],
       [6.78035941, 6.82074319, 0.        , 0.        ],
       [6.82074319, 6.7860502 , 0.        , 0.        ],
       [6.7860502 , 5.69803643, 0.        , 0.        ],
       [5.69803643, 5.4737329 , 0.        , 0.        ],
       [5.4737329 , 6.03630093, 0.        , 0.        ],
       [6.03630093, 5.29342261,

We have the co-variate matrix in the format $\left( Y_t, Y_{t - 1}, D_t, D_{t - 1} \right)$. We shall re-write it as $\left( y_n, y'_n, z_n \right)$ where $z$ is the indicator for death occurring at the second time point and $n$ equals the total number of $(t, t - 1)$ observations ranging over of all patients' tracks.

In [ ]:
temp = [None]*len(data)
for i in range(len(data)):
  y_values = [list(a) for a in zip(data[i][1][:-1], data[i][1][1:])]
  d_values = [list(a) for a in zip(data[i][2][:-1], data[i][2][1:])]
  temp[i] = np.concatenate((y_values, d_values), axis = 1)
data_regression = np.concatenate(tuple(a for a in temp), axis = 0)

In [ ]:
len(data_regression)

1922